# Knapsack With Integer Weights

Now we have weight $(w_\alpha \geq 0)$ and value $(c_\alpha \geq 0)$ of N items.
We put some of these into a knapsack we keep the limit of weight as $ \displaystyle W(= \sum _ {\alpha = 1} ^ {N} w_{\alpha}x_{\alpha})$ , $W_{limit}$ and take the maximum sum of value $ \displaystyle C(= \sum _ {\alpha = 1} ^ {N} c_{\alpha}x_{\alpha})$.

# Cost function

We have cost function from this paper.
( https://arxiv.org/abs/1302.5843 )。  

$H = H_A + H_B$  
$\displaystyle H_A = A \left( 1 - \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }+ A \left( \sum _ { i = 1 } ^ { W_{limit} } i y_i - \sum _ { \alpha } w_\alpha x_\alpha \right) ^ { 2 }$  
$\displaystyle H_B = -B \sum _ { \alpha } c_\alpha x_\alpha$  

# Creating QUBO
Now we have the QUBO

$H _ { A }$  
$ \displaystyle
=A \left\{
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right)
    +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i \right) ^ { 2 }
    +\left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) ^ { 2 }
    -2 \left( \sum _ { i = 1 } ^ { W_{limit} } iy_i \right) \left( \sum _ \alpha w_\alpha x_\alpha \right)
    +\left( \sum _ \alpha w_\alpha x_\alpha \right) ^ 2 \right\}
$  

$ \displaystyle
= A \left\{
    \left( \sum _ { i = 1 } ^ { W_{limit} } -2 y_i \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } y_i ^ 2 \right)
   +\left( \mathop { \sum \sum } _ { i \neq j } ^ { W_{limit} } 2 y_i y_j \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } i ^ 2 y_i ^ 2 \right)
   \right.
$  

$ \displaystyle
\quad \left.
   +\left( \mathop { \sum \sum } _ {i \neq j } ^ { W_{limit} } 2ij y_i y_j \right)
   +\left( \sum _ { i = 1 } ^ { W_{limit} } \sum _ { \alpha } \left( -2i w _ \alpha x _ \alpha y _ i \right) \right)
   +\left( \sum _ \alpha w_\alpha ^ 2 x_\alpha ^ 2 \right)
   +\left( \sum _ { \alpha } \sum _ { \beta } 2 w_\alpha w_\beta x_\alpha x_\beta\right )
   \right\}
$  

$ \displaystyle
= A \left\{
     \sum _ { \alpha } w_\alpha ^ 2x_\alpha ^ 2
   +\sum _ { \alpha } \sum _ { \beta } 2 w_\alpha w_\beta x_\alpha x_\beta
   +\sum _ { \alpha } \sum _ { i = 1 } ^ { W_{limit} }\left( -2 w_\alpha i \right) x_\alpha y_i
   +\sum _ { i = 1 } ^ { W_{limit} } \left( i ^ 2 - 1\right) y_i ^ 2
   +\mathop { \sum \sum } _ { i \neq j } ^ { W_{limit} } 2 \left( 1 + ij \right) y_i y_j
   \right\}
$ 

Let's start solving it using blueqat

pip install blueqat

And prepare for the libraries.

In [1]:
import numpy as np
from blueqat import vqe
from blueqat.pauli import X, Y, Z, I

Now we prepare some functions.


In [2]:
class Item():
    def __init__(self, number, weight, cost):
        self.__number = number
        self.__weight = weight
        self.__cost = cost

    @property
    def weight(self):
        return self.__weight

    @property
    def cost(self):
        return self.__cost

    def __str__(self):
        return f"#{self.__number} (weight : {self.weight}, cost : {self.cost})"

In [3]:
def get_qubo(items, wlimit, A, B):
    # qubo
    x_size = len(items)
    y_size = wlimit
    size = x_size + y_size
    qubo = 0
    for i in range(0,size):
        for j in range(0, size):
            if i > j:
                continue
                
            wi = items[i].weight if i < x_size else 0
            wj = items[j].weight if j < x_size else 0 
            ci = items[i].cost if i < x_size else 0      
            wsum_i = i - x_size + 1
            wsum_j = j - x_size + 1

            # diagonal elements
            if i == j:
                if i < x_size: # xi*xi
                    qubo += A*wi**2*Z(i)*Z(i) - B * ci
                else: # yi*yi
                    qubo += A * (-1 + wsum_i * wsum_j)*Z(i)*Z(i)
            # off-diagonal
            else: # i < j
                if i < x_size and j < x_size: # xi*xj
                    qubo += 2 * A * wi * wj*Z(i)*Z(j)
                elif i < x_size and j >= x_size: # xi*yj
                    qubo += -2 * A * wi * wsum_j*Z(i)*Z(j)
                else: # yi*yj
                    qubo += 2 * A * (1 + wsum_i * wsum_j)*Z(i)*Z(j)

    return qubo

Now we have some example of items.

In [4]:
items = []
items.append(Item(number=0, weight=1, cost=10))
items.append(Item(number=1, weight=2, cost=15))
items.append(Item(number=2, weight=2, cost=55))
items.append(Item(number=3, weight=3, cost=50))

And solve with some parameters.

In [5]:
from blueqat.pauli import X, Y, Z, I

# max weight
wlimit = 4

# get B from the cost
A = 1
cmax = max(items, key = lambda item : item.cost).cost
B = A / cmax * 0.9

step = 100

h = get_qubo(items, wlimit, A, B)

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()

In [6]:
print(result.most_common(20))

(((0, 0, 1, 0, 1, 1, 0, 0), 0.0382605841469839), ((1, 1, 0, 1, 0, 0, 1, 1), 0.0382605841469839), ((0, 1, 0, 0, 1, 1, 0, 0), 0.03826058414698384), ((1, 0, 1, 1, 0, 0, 1, 1), 0.03826058414698384), ((0, 0, 0, 1, 1, 1, 0, 0), 0.03463485420328392), ((1, 1, 1, 0, 0, 0, 1, 1), 0.03463485420328392), ((0, 1, 1, 0, 0, 1, 1, 0), 0.034626114860870374), ((1, 0, 0, 1, 1, 0, 0, 1), 0.034626114860870374), ((1, 1, 1, 0, 0, 1, 1, 0), 0.03273759172747863), ((0, 0, 0, 1, 1, 0, 0, 1), 0.03273759172747863), ((0, 0, 0, 1, 0, 1, 1, 0), 0.031446484135726795), ((1, 1, 1, 0, 1, 0, 0, 1), 0.031446484135726795), ((1, 0, 0, 0, 1, 1, 0, 0), 0.029891402021865947), ((0, 1, 1, 1, 0, 0, 1, 1), 0.029891402021865947), ((0, 0, 0, 0, 1, 0, 0, 0), 0.022638974869898073), ((1, 1, 1, 1, 0, 1, 1, 1), 0.022638974869898073), ((1, 0, 0, 0, 0, 1, 0, 0), 0.019124487517574347), ((0, 1, 1, 1, 1, 0, 1, 1), 0.019124487517574347), ((0, 1, 0, 0, 1, 0, 1, 0), 0.019071672930417224), ((1, 0, 1, 1, 0, 1, 0, 1), 0.019071672930417224))
